In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('HAM10000_metadata (2).csv')

In [3]:
data

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [4]:
counts = data['dx'].value_counts()
min_count = min(counts)

In [5]:
balanced_data = pd.DataFrame()
for label in counts.index:
    df = data[data['dx'] == label].sample(n=min_count, random_state=42)
    balanced_data = pd.concat([balanced_data, df])

In [6]:
balanced_data = balanced_data.sample(frac=1, random_state=42)
balanced_data.to_csv('HAM10000_balanced (2).csv', index=False)

In [7]:
balanced_data

,lesion_id,image_id,dx,dx_type,age,sex,localization
1965,HAM_0001444,ISIC_0024688,mel,histo,40.0,female,lower extremity
1139,HAM_0003290,ISIC_0030757,df,histo,80.0,male,upper extremity
2261,HAM_0002131,ISIC_0030901,mel,histo,55.0,male,upper extremity
9747,HAM_0005766,ISIC_0031738,akiec,histo,85.0,male,upper extremity
9795,HAM_0005505,ISIC_0026522,akiec,histo,50.0,male,upper extremity
...,...,...,...,...,...,...,...
4421,HAM_0001535,ISIC_0026653,nv,follow_up,45.0,male,trunk
5977,HAM_0007271,ISIC_0027862,nv,follow_up,30.0,male,abdomen
1035,HAM_0002034,ISIC_0027954,bkl,consensus,5.0,male,abdomen
2555,HAM_0005530,ISIC_0029899,bcc,histo,50.0,female,chest


In [8]:
balanced_data.size

5635

In [9]:
balanced_data.to_csv('HAM10000_balanced (2).csv', index=False)

In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

2023-04-16 18:47:21.395797: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 18:47:21.714811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-16 18:47:21.714846: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-16 18:47:22.784667: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [24]:
base_dir = "HAM10000_balanced"
df = pd.read_csv("HAM10000_balanced (2).csv")


In [25]:
balanced_data = balanced_data.rename(columns={"ID": "label"})

balanced_data =balanced_data.reset_index(drop=True)
balanced_data

,lesion_id,image_id,label,dx_type,age,sex,localization
0,HAM_0001444,ISIC_0024688,mel,histo,40.0,female,lower extremity
1,HAM_0003290,ISIC_0030757,df,histo,80.0,male,upper extremity
2,HAM_0002131,ISIC_0030901,mel,histo,55.0,male,upper extremity
3,HAM_0005766,ISIC_0031738,akiec,histo,85.0,male,upper extremity
4,HAM_0005505,ISIC_0026522,akiec,histo,50.0,male,upper extremity
...,...,...,...,...,...,...,...
800,HAM_0001535,ISIC_0026653,nv,follow_up,45.0,male,trunk
801,HAM_0007271,ISIC_0027862,nv,follow_up,30.0,male,abdomen
802,HAM_0002034,ISIC_0027954,bkl,consensus,5.0,male,abdomen
803,HAM_0005530,ISIC_0029899,bcc,histo,50.0,female,chest



import pandas as pd

images = []
labels = []


base_dir = "Images"

for i in range(len(balanced_data)):
    # Load the image and convert it to grayscale
    img_path = os.path.join(base_dir, balanced_data['image_id'][i]+ '.jpg')
    img = plt.imread(img_path)
    img = np.mean(img, axis=2)

   
    img = tf.image.resize(img, size=(256, 256))

    # Normalize the image pixel values to be between 0 and 1
    img = img / 255.0

    # Add the image and label to the lists
    images.append(img)
    labels.append(balanced_data['label'][i])

    # Convert the images and labels to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    # Convert the labels to one-hot encoded vectors
    labels = to_categorical(labels)


In [28]:
balanced_data['label'].unique()

array(['mel', 'df', 'akiec', 'bkl', 'nv', 'vasc', 'bcc'], dtype=object)

In [29]:
# label to number
balanced_data['label'] = balanced_data['label'].map({'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6})

In [30]:
# Preprocess the images
images = []
labels = []

base_dir = "Images"

for i in range(len(balanced_data)):
    try:
        # Load the image and convert it to grayscale
        img = plt.imread(base_dir + '/' + balanced_data['image_id'][i]+'.jpg')
        # img = np.mean(img, axis=2)
        print("=====================================")
        print(i)
        print(img.shape)

        # Resize the image to 256x256 pixels
        img = tf.image.resize(img, size=(256, 256))

        # Normalize the image pixel values to be between 0 and 1
        img = img / 255.0

        # Add the image and label to the lists
        images.append(img)
        labels.append(balanced_data['label'][i])
    except:
        pass

# Convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Convert the labels to one-hot encoded vectors
labels = to_categorical(labels)


0
(450, 600, 3)
4
(450, 600, 3)
5
(450, 600, 3)
7
(450, 600, 3)
8
(450, 600, 3)
9
(450, 600, 3)
11
(450, 600, 3)
15
(450, 600, 3)
18
(450, 600, 3)
19
(450, 600, 3)
21
(450, 600, 3)
22
(450, 600, 3)
24
(450, 600, 3)
26
(450, 600, 3)
27
(450, 600, 3)
28
(450, 600, 3)
29
(450, 600, 3)
30
(450, 600, 3)
34
(450, 600, 3)
35
(450, 600, 3)
36
(450, 600, 3)
37
(450, 600, 3)
38
(450, 600, 3)
46
(450, 600, 3)
50
(450, 600, 3)
51
(450, 600, 3)
53
(450, 600, 3)
54
(450, 600, 3)
60
(450, 600, 3)
66
(450, 600, 3)
67
(450, 600, 3)
70
(450, 600, 3)
73
(450, 600, 3)
75
(450, 600, 3)
76
(450, 600, 3)
83
(450, 600, 3)
84
(450, 600, 3)
85
(450, 600, 3)
86
(450, 600, 3)
87
(450, 600, 3)
88
(450, 600, 3)
90
(450, 600, 3)
92
(450, 600, 3)
93
(450, 600, 3)
94
(450, 600, 3)
96
(450, 600, 3)
97
(450, 600, 3)
98
(450, 600, 3)
101
(450, 600, 3)
103
(450, 600, 3)
104
(450, 600, 3)
108
(450, 600, 3)
109
(450, 600, 3)
111
(450, 600, 3)
112
(450, 600, 3)
115
(450, 600, 3)
117
(450, 600, 3)
118
(450, 600, 3)
120
(450, 

In [31]:
balanced_data['image_id']

0      ISIC_0024688
1      ISIC_0030757
2      ISIC_0030901
3      ISIC_0031738
4      ISIC_0026522
           ...     
800    ISIC_0026653
801    ISIC_0027862
802    ISIC_0027954
803    ISIC_0029899
804    ISIC_0030330
Name: image_id, Length: 805, dtype: object

In [32]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [39]:
labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [40]:
# Define the CNN model
inputs = Input(shape=(256, 256, 3))

x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.25)(x)

x = Flatten()(x)

x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.5)(x)

outputs = Dense(7, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)


In [41]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


Epoch 1/10
11/11 [==============================] - 43s 4s/step - loss: 11.1774 - accuracy: 0.1385 - val_loss: 1.9454 - val_accuracy: 0.1951
Epoch 2/10
 6/11 [===============>..............] - ETA: 17s - loss: 1.9477 - accuracy: 0.1354

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np